In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import time as time

In [2]:
pdb=app.PDBFile('TiO21aHigher1.pdb')
forcefield=mm.app.ForceField('TiO2_imcomplete.xml')


C:\Users\Garrett\Anaconda3\lib\site-packages\simtk\openmm\app\internal\pdbstructure.py:536: UserWarning: WARNING: duplicate atom (HETATM    3  O           0       0.000  -1.362  -0.573  1.00  0.00           O  , HETATM    2  O           0       0.000   1.362  -0.573  1.00  0.00           O  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))


In [3]:
system=forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff)
integrator=mm.LangevinIntegrator(208.15*unit.kelvin, 5.0/unit.picoseconds, 1.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform=mm.Platform.getPlatformByName('Reference')
simulation=app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [4]:
print('minimizing...')

st=simulation.context.getState(getPositions=True,getEnergy=True)
print('PE b4 mim is %s' %(st.getPotentialEnergy()))

simulation.minimizeEnergy(maxIterations=100)

st=simulation.context.getState(getPositions=True,getEnergy=True)
print("PE after mim 4 running is %s" %(st.getPotentialEnergy()))

minimizing...
PE b4 mim is 0.00043470359749896815 kJ/mol
PE after mim 4 running is 2.4199383258342844e-05 kJ/mol


In [5]:
print('Equibiration...')

simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True,
                                                 potentialEnergy=True, temperature=True, separator='\t'))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equibiration...
#"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"
250	4.0575924970596144	157.9128628179142
500	2.1355775968983344	231.21034915271468
750	0.8528530209016691	53.12903860029202
1000	3.338149857557177	184.67766351342814
1250	1.2193978178196219	178.02482754913243
1500	0.3812921196346344	176.57440271023776
1750	6.335734727225286	222.71679674757306
2000	1.8700105868431536	107.2920802679919
2250	4.5569515960572105	87.29796265640762
2500	0.022740703542477755	298.4365556112495
2750	1.1199229880082389	244.23117898041806
3000	1.3533125967720725	77.06230646510755
3250	0.5771777731129618	148.642741402509
3500	0.17456046448460716	414.9739144598807
3750	4.312651406847732	75.14572285195564
4000	0.7308134872073713	130.35683193164442
4250	3.1708665744017503	110.56749354649514
4500	0.9138537056241387	124.23277904911816
4750	7.8960255894878415	128.82449450887026
5000	1.3463966825775602	413.32014399873356


In [6]:
tinit=time.time()
simulation.reporters.clear()
simulation.reporters.append(app.StateDataReporter(stdout,50000, 
                                                  step=True, 
                                                  time=True, 
                                                  potentialEnergy=True, 
                                                  temperature=True, 
                                                 speed=True, 
                                                 separator='\t'))
simulation.reporters.append(app.DCDReporter('TiO21a.dcd', 100))

simulation.step(20000000)

tfinal=time.time()

print('Done!')
print('Time required for simualtion:', tfinal-tinit, 'seconds')

#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
50000	49.99999999997417	1.3967702147681402	254.81473209348883	--
100000	100.00000000011343	3.1778303446897405	154.0514432590494	2.21e+04
150000	150.00000000035217	6.857354542403396	133.46801740081239	2.43e+04
200000	200.00000000059092	7.017642395468188	220.20785082454955	2.67e+04
250000	250.00000000082966	0.421501181832314	144.63283779966585	2.88e+04
300000	299.9999999998178	1.2364547438810491	207.5151957080095	2.92e+04
350000	349.9999999986355	5.843524094272233	188.70407516083966	3e+04
400000	399.99999999745313	9.104503608546102	76.88619544151675	2.85e+04
450000	449.9999999962708	1.363149179193988	467.79700499827726	2.85e+04
500000	499.99999999508844	2.373251983658595	81.079910438836	2.82e+04
550000	549.9999999939062	2.0393298837813205	543.7674199798727	2.87e+04
600000	599.9999999927238	2.156021734636419	310.78080077225945	2.94e+04
650000	649.9999999915415	2.953991304159092	271.50274504127907	3e+04
700

5700000	5700.000000702493	6.926367651318075	189.36695509410498	3.18e+04
5750000	5750.000000712679	0.7114440765518038	148.99750511657538	3.18e+04
5800000	5800.0000007228655	5.513553147596487	116.61383321612449	3.19e+04
5850000	5850.000000733052	5.7215692041020505	845.211103249295	3.19e+04
5900000	5900.000000743238	0.18855692267128818	162.95718650367155	3.19e+04
5950000	5950.0000007534245	1.1203027095605487	200.9179917787674	3.19e+04
6000000	6000.000000763611	1.933325264980679	72.46796055783408	3.19e+04
6050000	6050.000000773797	0.23220395925418064	210.24567996657285	3.2e+04
6100000	6100.0000007839835	2.220511909338361	344.3098274448256	3.19e+04
6150000	6150.00000079417	15.926311801452389	113.79710778836066	3.2e+04
6200000	6200.000000804356	1.9386141793931166	131.3612540826705	3.2e+04
6250000	6250.000000814543	1.43567024908643	160.88090934830478	3.2e+04
6300000	6300.000000824729	1.2885079514169115	350.79840732862624	3.2e+04
6350000	6350.000000834915	1.406807900046966	23.341515861031798	3

11400000	11400.000001863735	4.400643422065716	301.14787956735773	3.12e+04
11450000	11450.000001873921	8.506290916307298	86.18365984966599	3.11e+04
11500000	11500.000001884107	1.68765911368215	268.534418616837	3.11e+04
11550000	11550.000001894294	4.91557320503787	246.23029652171655	3.11e+04
11600000	11600.00000190448	0.4880625587682938	229.76946199469927	3.1e+04
11650000	11650.000001914666	4.617088851697672	186.81599425528063	3.1e+04
11700000	11700.000001924853	1.0665611323195463	95.19184177246161	3.1e+04
11750000	11750.00000193504	2.520956144879309	414.1942618984624	3.1e+04
11800000	11800.000001945225	1.6741095361346157	98.0662682325489	3.09e+04
11850000	11850.000001955412	1.8409633096946911	158.31382337877866	3.09e+04
11900000	11900.000001965598	1.6668588897725338	245.2636315217292	3.09e+04
11950000	11950.000001975784	0.3444710923371528	185.2007286444626	3.08e+04
12000000	12000.00000198597	6.092463659223281	112.81266968239417	3.08e+04
12050000	12050.000001996157	0.7169437555807783	119

17000000	17000.000003004603	3.8612111573644046	247.4244756315847	2.75e+04
17050000	17050.00000301479	1.4729439018821844	270.92236132223894	2.75e+04
17100000	17100.000003024976	7.0769043309104145	470.9661686709571	2.75e+04
17150000	17150.000003035162	1.3192866594216441	98.92356828279414	2.75e+04
17200000	17200.00000304535	0.71553023140179	169.07298299236066	2.75e+04
17250000	17250.000003055535	0.6678053316050673	361.42636053790636	2.75e+04
17300000	17300.00000306572	1.845715180562174	223.26257570424374	2.75e+04
17350000	17350.000003075907	2.0489127037458124	127.56538929322379	2.75e+04
17400000	17400.000003086094	1.0839776699864292	237.33545763661567	2.75e+04
17450000	17450.00000309628	0.4836800806726493	162.2461281038693	2.75e+04
17500000	17500.000003106466	3.2406788537536837	49.04732801738977	2.74e+04
17550000	17550.000003116653	1.5203394931106131	236.842444335064	2.74e+04
17600000	17600.00000312684	2.8510142981860134	217.38092336145576	2.74e+04
17650000	17650.000003137025	6.1994906840